In [1]:
##############################從右邊拼

import cv2
import numpy as np

cap    = cv2.VideoCapture('hw2_dataset1.mp4')
print('height:{} width:{}'.format(int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))))
frame_num = 0
total_frame=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

def set_frame_number(x):
    global frame_num
    frame_num = x
    return

result = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//4,int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//4*3,3))
count  = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//4,int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//4*3))
ones = np.ones(((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//4,int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//4)))

cv2.namedWindow('matching')
cv2.createTrackbar('frame no.','matching',0,total_frame-1,set_frame_number)

kpdetector = cv2.xfeatures2d.SIFT_create() 
#kpdetector = cv2.AKAZE_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

while frame_num < total_frame and frame_num < 95:
    cv2.setTrackbarPos('frame no.','matching',frame_num)
    cap.set(cv2.CAP_PROP_POS_FRAMES,frame_num)
    ret, frame2 = cap.read() 
    if ret==False:
        break    
    
    frame2 = cv2.resize(frame2,(frame2.shape[1]//4,frame2.shape[0]//4))
    
#kp1, dt1 = kpdetector.detectAndCompute(frame1,None)
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    kp2 = kpdetector.detect(gray,None)
    dt2 = kpdetector.compute(gray,kp2)[1]
    if frame_num == 0:
        T      = np.eye(3)
        T[0,2] = result.shape[1]-frame2.shape[1]
        T[1,2] = 0
        result = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count= cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        count += t_count.astype(np.float)
        disp = result.copy()
        cv2.imshow('stitched image',disp.astype(np.uint8))
    
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2
    else:
        # Match descriptors.
        matches = bf.match(dt2,dt1)
        print('{}, # of matches:{}'.format(frame_num,len(matches)))

        # Sort in ascending order of distance.
        matches = sorted(matches, key = lambda x:x.distance)
        
        src = []
        dst = []
        for m in matches:
            src.append(kp2[m.queryIdx].pt + (1,))
            dst.append(kp1[m.trainIdx].pt + (1,))
            
        src = np.array(src,dtype=np.float)
        dst = np.array(dst,dtype=np.float)
    
        # find a homography to map src to dst
        A, mask = cv2.findHomography(src, dst, cv2.RANSAC) 
        
        # map to the first frame
        T = T.dot(A)
        warp_img = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count  = cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        result+= warp_img
        count += t_count.astype(np.float)

        t_count= count.copy()
        t_count[t_count == 0] = 1
        disp = result.copy()
        
        disp[:,:,0] = result[:,:,0] / t_count
        disp[:,:,1] = result[:,:,1] / t_count
        disp[:,:,2] = result[:,:,2] / t_count
 
        cv2.imshow('stitched image',disp.astype(np.uint8))
   
        cv2.imshow('matching',cv2.drawMatches(frame2,kp2,frame1,kp1,matches[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        
        
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2

    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    frame_num += 1
cv2.waitKey()    
cap.release()
cv2.destroyAllWindows()

height:1080 width:1920
87, # of matches:434
86, # of matches:449
85, # of matches:435
84, # of matches:430
83, # of matches:441
82, # of matches:443
81, # of matches:419
80, # of matches:432
79, # of matches:427
78, # of matches:445
77, # of matches:434
76, # of matches:444
75, # of matches:476
74, # of matches:455
73, # of matches:439
72, # of matches:431
71, # of matches:427
70, # of matches:472
69, # of matches:446
68, # of matches:455
67, # of matches:430
66, # of matches:424
65, # of matches:409
64, # of matches:416
63, # of matches:431
62, # of matches:447
61, # of matches:442
60, # of matches:449
59, # of matches:445
58, # of matches:426
57, # of matches:418
56, # of matches:430
55, # of matches:440
54, # of matches:419
53, # of matches:410
52, # of matches:416
51, # of matches:401
50, # of matches:403
49, # of matches:416
48, # of matches:449
47, # of matches:428
46, # of matches:419
45, # of matches:430
44, # of matches:462
43, # of matches:467
42, # of matches:435
41, # of ma

In [2]:
#################################從中間拼
import cv2
import numpy as np

cap    = cv2.VideoCapture('hw2_dataset1.mp4')
print('height:{} width:{}'.format(int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))))
frame_num = 0
total_frame=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

goal = 'splice_video.mp4'
width = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))//4*3
height = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))//4
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter(goal, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), fps, (width, height))

def set_frame_number(x):
    global frame_num
    frame_num = x
    return

result = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//4,int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//4*3,3))
count  = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//4,int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//4*3))
ones = np.ones(((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//4,int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//4)))

cv2.namedWindow('matching')
cv2.createTrackbar('frame no.','matching',0,total_frame-1,set_frame_number)

kpdetector = cv2.xfeatures2d.SIFT_create() 
#kpdetector = cv2.AKAZE_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)


########################往右邊拼###########################

frame_num = total_frame // 2
while frame_num > 0:
    cv2.setTrackbarPos('frame no.','matching',frame_num)
    cap.set(cv2.CAP_PROP_POS_FRAMES,frame_num)
    ret, frame2 = cap.read() 
    if ret==False:
        break    
    
    frame2 = cv2.resize(frame2,(frame2.shape[1]//4,frame2.shape[0]//4))
    
    # kp1, dt1 = kpdetector.detectAndCompute(frame1,None)
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    kp2 = kpdetector.detect(gray,None)
    dt2 = kpdetector.compute(gray,kp2)[1]
    if frame_num == total_frame // 2:
        T      = np.eye(3)
        T[0,2] = (result.shape[1]-frame2.shape[1]) / 2 - 190 ##取中間值再做點誤差修正
        T[1,2] = 0
        result = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count= cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        count += t_count.astype(np.float)
        disp = result.copy()
        cv2.imshow('stitched image',disp.astype(np.uint8))
    
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2
    else:
        # Match descriptors.
        matches = bf.match(dt2,dt1)
        print('{}, # of matches:{}'.format(frame_num,len(matches)))

        # Sort in ascending order of distance.
        matches = sorted(matches, key = lambda x:x.distance)
        
        src = []
        dst = []
        for m in matches:
            src.append(kp2[m.queryIdx].pt + (1,))
            dst.append(kp1[m.trainIdx].pt + (1,))
            
        src = np.array(src,dtype=np.float)
        dst = np.array(dst,dtype=np.float)
    
        # find a homography to map src to dst
        A, mask = cv2.findHomography(src, dst, cv2.RANSAC) 
        
        # map to the first frame
        T = T.dot(A)
        warp_img = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count  = cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        result+= warp_img
        count += t_count.astype(np.float)

        t_count= count.copy()
        t_count[t_count == 0] = 1
        disp = result.copy()
        
        disp[:,:,0] = result[:,:,0] / t_count
        disp[:,:,1] = result[:,:,1] / t_count
        disp[:,:,2] = result[:,:,2] / t_count
 
        cv2.imshow('stitched image',disp.astype(np.uint8))
   
        cv2.imshow('matching',cv2.drawMatches(frame2,kp2,frame1,kp1,matches[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        
        
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2
        
    videoWriter.write(disp.astype(np.uint8))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    frame_num -= 1


########################往左邊拼###########################

frame_num = total_frame // 2
while frame_num < total_frame:
    cv2.setTrackbarPos('frame no.','matching',frame_num)
    cap.set(cv2.CAP_PROP_POS_FRAMES,frame_num)
    ret, frame2 = cap.read() 
    if ret==False:
        break    
    
    frame2 = cv2.resize(frame2,(frame2.shape[1]//4,frame2.shape[0]//4))
    
    # kp1, dt1 = kpdetector.detectAndCompute(frame1,None)
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    kp2 = kpdetector.detect(gray,None)
    dt2 = kpdetector.compute(gray,kp2)[1]
    if frame_num ==  total_frame // 2:
        T      = np.eye(3)
        T[0,2] =  (result.shape[1]-frame2.shape[1])/2 - 190 ##取中間值再做點誤差修正
        T[1,2] = 0
        t_count= cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)

        count += t_count.astype(np.float)
        disp = result.copy()
        cv2.imshow('stitched image',disp.astype(np.uint8))
    
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2
        
    else:
        # Match descriptors.
        matches = bf.match(dt2,dt1)
        print('{}, # of matches:{}'.format(frame_num,len(matches)))

        # Sort in ascending order of distance.
        matches = sorted(matches, key = lambda x:x.distance)
        
        src = []
        dst = []
        for m in matches:
            src.append(kp2[m.queryIdx].pt + (1,))
            dst.append(kp1[m.trainIdx].pt + (1,))
            
        src = np.array(src,dtype=np.float)
        dst = np.array(dst,dtype=np.float)
    
        # find a homography to map src to dst
        A, mask = cv2.findHomography(src, dst, cv2.RANSAC) 
        
        # map to the first frame
        T = T.dot(A)
        warp_img = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count  = cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        result+= warp_img
        count += t_count.astype(np.float)

        t_count= count.copy()
        t_count[t_count == 0] = 1
        disp = result.copy()
        
        disp[:,:,0] = result[:,:,0] / t_count
        disp[:,:,1] = result[:,:,1] / t_count
        disp[:,:,2] = result[:,:,2] / t_count
 
        cv2.imshow('stitched image',disp.astype(np.uint8))
   
        cv2.imshow('matching',cv2.drawMatches(frame2,kp2,frame1,kp1,matches[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        
        
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2
    
    videoWriter.write(disp.astype(np.uint8))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    frame_num += 1
    
cv2.waitKey()    
cap.release()
cv2.destroyAllWindows()
videoWriter.release()

print("end")

height:1080 width:1920
87, # of matches:434
86, # of matches:449
85, # of matches:435
84, # of matches:430
83, # of matches:441
82, # of matches:443
81, # of matches:419
80, # of matches:432
79, # of matches:427
78, # of matches:445
77, # of matches:434
76, # of matches:444
75, # of matches:476
74, # of matches:455
73, # of matches:439
72, # of matches:431
71, # of matches:427
70, # of matches:472
69, # of matches:446
68, # of matches:455
67, # of matches:430
66, # of matches:424
65, # of matches:409
64, # of matches:416
63, # of matches:431
62, # of matches:447
61, # of matches:442
60, # of matches:449
59, # of matches:445
58, # of matches:426
57, # of matches:418
56, # of matches:430
55, # of matches:440
54, # of matches:419
53, # of matches:410
52, # of matches:416
51, # of matches:401
50, # of matches:403
49, # of matches:416
48, # of matches:449
47, # of matches:428
46, # of matches:419
45, # of matches:430
44, # of matches:462
43, # of matches:467
42, # of matches:435
41, # of ma

In [7]:
##############################從右邊拼，拼照片
import cv2
import numpy as np

frame_num = 0
total_frame = 81

goal = 'splice_dataset1.mp4'
width = (1920)//4*2
height = (1080)//4
fps = 30
videoWriter = cv2.VideoWriter(goal, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), fps, (width, height))


def set_frame_number(x):
    global frame_num
    frame_num = x
    return

result = np.zeros((int(1080)//4,int(1920)//4*2,3))
count  = np.zeros((int(1080)//4,int(1920)//4*2))
ones = np.ones(((int(1080)//4,int(1920)//4)))

cv2.namedWindow('matching')
cv2.createTrackbar('frame no.','matching',0,total_frame-1,set_frame_number)

kpdetector = cv2.xfeatures2d.SIFT_create() 
#kpdetector = cv2.AKAZE_create()

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
i = 0
while i in range(81):
    frame2=cv2.imread(".\\dataset1\\dataset_1_%d.jpg"%(i))
    print(".\\dataset1\\dataset_1_%d.jpg"%(i))
    frame2 = cv2.resize(frame2,(frame2.shape[1]//4,frame2.shape[0]//4))
    
#kp1, dt1 = kpdetector.detectAndCompute(frame1,None)
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    kp2 = kpdetector.detect(gray,None)
    dt2 = kpdetector.compute(gray,kp2)[1]
    if frame_num == 0:
        T      = np.eye(3)
        T[0,2] = result.shape[1]-frame2.shape[1]
        T[1,2] = 0
        result = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count= cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        count = t_count.astype(np.float)
        disp = result.copy()
        cv2.imshow('stitched image',disp.astype(np.uint8))
    
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2
    else:
        # Match descriptors.
        matches = bf.match(dt2,dt1)
        print('{}, # of matches:{}'.format(frame_num,len(matches)))

        # Sort in ascending order of distance.
        matches = sorted(matches, key = lambda x:x.distance)
        
        src = []
        dst = []
        for m in matches:
            src.append(kp2[m.queryIdx].pt + (1,))
            dst.append(kp1[m.trainIdx].pt + (1,))
            
        src = np.array(src,dtype=np.float)
        dst = np.array(dst,dtype=np.float)
    
        # find a homography to map src to dst
        A, mask = cv2.findHomography(src, dst, cv2.RANSAC) 
        
        # map to the first frame
        T = T.dot(A)
        warp_img = cv2.warpPerspective(frame2,T,(result.shape[1],result.shape[0])).astype(np.float)
        t_count  = cv2.warpPerspective(ones,T,(result.shape[1],result.shape[0])).astype(np.float)
        result+= warp_img
        count += t_count.astype(np.float)

        t_count= count.copy()
        t_count[t_count == 0] = 1
        disp = result.copy()
        
        disp[:,:,0] = result[:,:,0] / t_count
        disp[:,:,1] = result[:,:,1] / t_count
        disp[:,:,2] = result[:,:,2] / t_count

        cv2.imshow('stitched image',disp.astype(np.uint8))
   
        cv2.imshow('matching',cv2.drawMatches(frame2,kp2,frame1,kp1,matches[:15], None, flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS))
        
        
        frame1 = frame2
        kp1 = kp2
        dt1 = dt2
    
    videoWriter.write(disp.astype(np.uint8))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        break
    frame_num += 1
    i+=1
cv2.waitKey()    
videoWriter.release()
cv2.destroyAllWindows()
print("end")

.\dataset1\dataset_1_0.jpg
.\dataset1\dataset_1_1.jpg
1, # of matches:463
.\dataset1\dataset_1_2.jpg
2, # of matches:441
.\dataset1\dataset_1_3.jpg
3, # of matches:440
.\dataset1\dataset_1_4.jpg
4, # of matches:392
.\dataset1\dataset_1_5.jpg
5, # of matches:394
.\dataset1\dataset_1_6.jpg
6, # of matches:394
.\dataset1\dataset_1_7.jpg
7, # of matches:410
.\dataset1\dataset_1_8.jpg
8, # of matches:401
.\dataset1\dataset_1_9.jpg
9, # of matches:395
.\dataset1\dataset_1_10.jpg
10, # of matches:421
.\dataset1\dataset_1_11.jpg
11, # of matches:412
.\dataset1\dataset_1_12.jpg
12, # of matches:429
.\dataset1\dataset_1_13.jpg
13, # of matches:421
.\dataset1\dataset_1_14.jpg
14, # of matches:430
.\dataset1\dataset_1_15.jpg
15, # of matches:444
.\dataset1\dataset_1_16.jpg
16, # of matches:435
.\dataset1\dataset_1_17.jpg
17, # of matches:409
.\dataset1\dataset_1_18.jpg
18, # of matches:410
.\dataset1\dataset_1_19.jpg
19, # of matches:422
.\dataset1\dataset_1_20.jpg
20, # of matches:422
.\dataset1\

In [6]:
import cv2
import numpy as np
   
frame2=cv2.imread(".\\dataset1\\dataset_1_%d.jpg"%(i))#路径自己选择择
print(frame2.shape)
#frame2=cv2.imread("C:\\Users\\peggy\\OneDrive\\桌面\\NTOU\\機器視覺\\作業二\\dataset1\\dataset_1_0.jpg")#路径自己选择择
frame2 = cv2.resize(frame2,(frame2.shape[1]//4,frame2.shape[0]//4))
#print("C:\\Users\\peggy\\OneDrive\\桌面\\NTOU\\機器視覺\\作業二\\dataset1\\dataset_1_%d.jpg"%(i))
cv2.imshow('hello',frame2)
    
cv2.waitKey()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'